In [1]:
import math
import torch
from torch import empty , cat , arange
from torch . nn . functional import fold , unfold

#to delete
from torch import nn
from others.modules import *


In [22]:
# Put on GPU
device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
# Extract images

noisy_imgs_1 , noisy_imgs_2 = torch.load('data/train_data.pkl')
noisy_imgs , clean_imgs = torch.load ('data/val_data.pkl')

noisy_imgs = noisy_imgs/255
clean_imgs = clean_imgs/255

# select a preset of images:

imgs_1 = noisy_imgs_1[:1]/255
imgs_2 = noisy_imgs_2[:10000]/255
print(imgs_1.shape)

torch.Size([1, 3, 32, 32])


In [2]:
class Model(object):
    def __init__(self):
        self.sequence = Sequential(Convolution(3, 3, kernel_size = 2, stride = 2),
                                    ReLU(),
                                    Upsample(2),
                                    Sigmoid())

        self.criterion = MSE()
        print('Model class initialisation')

    def train(self,train_input,train_target):
    
        predict_img = self.sequence.forward(train_input)
        loss = self.criterion.forward(predict_img, train_target)

        backward_img = self.sequence.backward(img)

        return backward_img
        

In [3]:
img = torch.randn(1, 3, 32, 32)
img_error = torch.randn(1, 3, 32, 32)

model = Model()

output = model.train(img, img_error)

print(output.shape)



Model class initialisation
torch.Size([1, 3, 32, 32])
